In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'sage-dragon-413617' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=sage-dragon-413617:US:bquxjob_1ff9bd77_18e26d6e23d)
back to BigQuery to edit the query within the BigQuery user interface.

In [2]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_1ff9bd77_18e26d6e23d') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT * 
FROM `bigquery-public-data.google_ads_transparency_center.creative_stats` 
LIMIT 100000


# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [3]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1ff9bd77_18e26d6e23d') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,advertiser_id,creative_id,creative_page_url,ad_format_type,advertiser_disclosed_name,advertiser_legal_name,advertiser_location,advertiser_verification_status,region_stats,audience_selection_approach_info,topic
0,AR06101118588632956929,CR17599147911524909057,https://adstransparency.google.com/advertiser/...,VIDEO,Hih Invest Eskilstuna AB (Tuna Park),,,UNVERIFIED,"[{'region_code': 'EEA', 'first_shown': '2023-1...","{'demographic_info': 'CRITERIA_UNUSED', 'geo_l...",Commercial
1,AR13529253531948679169,CR10607723043843211265,https://adstransparency.google.com/advertiser/...,VIDEO,Äventyrshuset Bborg Ab,,,UNVERIFIED,"[{'region_code': 'EEA', 'first_shown': '2024-0...","{'demographic_info': 'CRITERIA_UNUSED', 'geo_l...",Commercial
2,AR08593965848304025601,CR01424717927134789633,https://adstransparency.google.com/advertiser/...,VIDEO,Vakantie Discounter NL,,,UNVERIFIED,"[{'region_code': 'EEA', 'first_shown': '2023-1...","{'demographic_info': 'CRITERIA_UNUSED', 'geo_l...",Commercial
3,AR04183151120983523329,CR12894387546611515393,https://adstransparency.google.com/advertiser/...,TEXT,Last Minute doo,Last Minute doo,"""RS""",VERIFIED,"[{'region_code': 'BG', 'first_shown': '2023-12...","{'demographic_info': 'CRITERIA_INCLUDED', 'geo...",Travel & Tourism
4,AR02997330338688008193,CR09642303627942952961,https://adstransparency.google.com/advertiser/...,TEXT,AD TYRES INTERNATIONAL SLU,AD TYRES INTERNATIONAL SLU,AD,VERIFIED,"[{'region_code': 'EEA', 'first_shown': '2023-0...","{'demographic_info': 'CRITERIA_INCLUDED', 'geo...",Autos & Vehicles
...,...,...,...,...,...,...,...,...,...,...,...
99995,AR16138687197833330689,CR01824111368396079105,https://adstransparency.google.com/advertiser/...,TEXT,Kayak Europe GmbH,Kayak Europe GmbH,CH,VERIFIED,"[{'region_code': 'EEA', 'first_shown': '2023-0...","{'demographic_info': 'CRITERIA_INCLUDED', 'geo...",Travel & Tourism
99996,AR16138687197833330689,CR04675465801736650753,https://adstransparency.google.com/advertiser/...,TEXT,Kayak Europe GmbH,Kayak Europe GmbH,CH,VERIFIED,"[{'region_code': 'EEA', 'first_shown': '2023-0...","{'demographic_info': 'CRITERIA_INCLUDED', 'geo...",Travel & Tourism
99997,AR16138687197833330689,CR11824840462550695937,https://adstransparency.google.com/advertiser/...,TEXT,Kayak Europe GmbH,Kayak Europe GmbH,CH,VERIFIED,"[{'region_code': 'EEA', 'first_shown': '2023-0...","{'demographic_info': 'CRITERIA_INCLUDED', 'geo...",Travel & Tourism
99998,AR16138687197833330689,CR03216588384789069825,https://adstransparency.google.com/advertiser/...,TEXT,Kayak Europe GmbH,Kayak Europe GmbH,CH,VERIFIED,"[{'region_code': 'EEA', 'first_shown': '2023-0...","{'demographic_info': 'CRITERIA_INCLUDED', 'geo...",Travel & Tourism


In [4]:
import pandas as pd

# Explode 'region_stats' column to create separate rows for each dictionary
exploded_results = results.explode('region_stats')

# Normalize the exploded DataFrame to flatten the dictionaries within 'region_stats'
normalized_results = pd.json_normalize(exploded_results['region_stats'])

# Create a new DataFrame by converting the 'audience_selection_approach_info' column to a DataFrame
audience_info_df = pd.DataFrame(results['audience_selection_approach_info'].tolist())

# Concatenate the original DataFrame with the normalized DataFrame and audience_info_df
merged_results = pd.concat([exploded_results.reset_index(drop=True), normalized_results.reset_index(drop=True), audience_info_df.reset_index(drop=True)], axis=1)

# Drop the original 'region_stats' and 'audience_selection_approach_info' columns
merged_results = merged_results.drop(columns=['region_stats', 'audience_selection_approach_info'])

# Display the merged results
print("Merged Results:")
merged_results.head()


Merged Results:


,advertiser_id,creative_id,creative_page_url,ad_format_type,advertiser_disclosed_name,advertiser_legal_name,advertiser_location,advertiser_verification_status,topic,region_code,...,times_shown_upper_bound,times_shown_start_date,times_shown_availability_date,surface_serving_stats.surface_serving_stats,surface_serving_stats,demographic_info,geo_location,contextual_signals,customer_lists,topics_of_interest
0,AR06101118588632956929,CR17599147911524909057,https://adstransparency.google.com/advertiser/...,VIDEO,Hih Invest Eskilstuna AB (Tuna Park),,,UNVERIFIED,Commercial,EEA,...,20000.0,2023-10-22,None,"[{'surface': 'MAPS', 'times_shown_upper_bound'...",NaN,CRITERIA_UNUSED,CRITERIA_UNUSED,CRITERIA_INCLUDED,CRITERIA_UNUSED,CRITERIA_UNUSED
1,AR06101118588632956929,CR17599147911524909057,https://adstransparency.google.com/advertiser/...,VIDEO,Hih Invest Eskilstuna AB (Tuna Park),,,UNVERIFIED,Commercial,SE,...,20000.0,2023-10-22,None,"[{'surface': 'SEARCH', 'times_shown_upper_boun...",NaN,CRITERIA_UNUSED,CRITERIA_UNUSED,CRITERIA_INCLUDED,CRITERIA_UNUSED,CRITERIA_UNUSED
2,AR13529253531948679169,CR10607723043843211265,https://adstransparency.google.com/advertiser/...,VIDEO,Äventyrshuset Bborg Ab,,,UNVERIFIED,Commercial,EEA,...,NaN,None,2024-05-04,NaN,NaN,CRITERIA_UNUSED,CRITERIA_UNUSED,CRITERIA_INCLUDED_AND_EXCLUDED,CRITERIA_UNUSED,CRITERIA_UNUSED
3,AR13529253531948679169,CR10607723043843211265,https://adstransparency.google.com/advertiser/...,VIDEO,Äventyrshuset Bborg Ab,,,UNVERIFIED,Commercial,SE,...,NaN,None,2024-05-04,NaN,NaN,CRITERIA_INCLUDED,CRITERIA_INCLUDED,CRITERIA_INCLUDED,CRITERIA_UNUSED,CRITERIA_UNUSED
4,AR08593965848304025601,CR01424717927134789633,https://adstransparency.google.com/advertiser/...,VIDEO,Vakantie Discounter NL,,,UNVERIFIED,Commercial,EEA,...,NaN,None,2024-03-30,NaN,NaN,CRITERIA_INCLUDED,CRITERIA_INCLUDED_AND_EXCLUDED,CRITERIA_INCLUDED_AND_EXCLUDED,CRITERIA_UNUSED,CRITERIA_UNUSED


In [ ]:
from google.colab import files

# Save the merged dataframe as an Excel file
merged_results.to_excel('creative_stats.xlsx', index=False)

# Download the Excel file
files.download('creative_stats.xlsx')
